In [51]:
from category_encoders.ordinal import OrdinalEncoder

#데이터불러올때 데이터타입정해주기
#?를 처리해서 차원을 줄이자
#데이터의 출처를 알아봐라
import numpy as np
import pandas as pd

In [52]:
train = pd.read_csv('./data/kakr-4th-competition/train.csv')
test = pd.read_csv('./data/kakr-4th-competition/test.csv')

In [53]:
sample_submission = pd.read_csv('./data/kakr-4th-competition/sample_submission.csv')

In [54]:
train.drop('id', axis=1, inplace = True)
test.drop('id', axis=1, inplace = True)

### 파생변수 추가

In [56]:
def portion_income(column):
    train[column] = train[column].astype('str')
    col_lst = train[column].unique().tolist()
    df = train[[column,'income']]

    for x in col_lst:
        locals()["high" + '_' + x] = df[(df[column] == x) & (df.income == ">50K")]
        locals()["low" + '_' + x]  = df[(df[column] == x) & (df.income == "<=50K")]
    col_name = []
    col_name_value = []
    for x in col_lst:
        col_name.append(x)
        col_name_value.append(len(locals()["high" + '_' + x]) / (len(locals()["high" + '_' + x]) + len(locals()["low" + '_' + x])))
        #print(x , len(locals()["high" + '_' + x]) / (len(locals()["high" + '_' + x]) + len(locals()["low" + '_' + x])))
    data = pd.DataFrame({"name" : col_name, "value" : col_name_value }).sort_values(by = 'value', ascending = False)
    return data

In [57]:
education_por = portion_income('education')
education_por.columns = ['education', 'edu_portion']
train = pd.merge(train, education_por, on='education', how='left')
test = pd.merge(test, education_por, on='education', how='left')

In [58]:
workclass_por = portion_income('workclass')
workclass_por.columns = ['workclass', 'wor_portion']
train = pd.merge(train, workclass_por, on='workclass', how='left')
test = pd.merge(test, workclass_por, on='workclass', how='left')

In [59]:
race_por = portion_income('race')
race_por.columns = ['race', 'race_portion']
train = pd.merge(train, race_por, on='race', how='left')
test = pd.merge(test, race_por, on='race', how='left')

In [60]:
native_por = portion_income('native_country')
native_por.columns = ['native_country', 'native_portion']
train = pd.merge(train, native_por, on='native_country', how='left')
test = pd.merge(test, native_por, on='native_country', how='left')

marital_por = portion_income('marital_status')
marital_por.columns = ['marital_status', 'marital_portion']
train = pd.merge(train, marital_por, on='marital_status', how='left')
test = pd.merge(test, marital_por, on='marital_status', how='left')

occupation_por = portion_income('occupation')
occupation_por.columns = ['occupation', 'occupation_portion']
train = pd.merge(train, occupation_por, on='occupation', how='left')
test = pd.merge(test, occupation_por, on='occupation', how='left')

relationship_por = portion_income('relationship')
relationship_por.columns = ['relationship', 'relationship_portion']
train = pd.merge(train, relationship_por, on='relationship', how='left')
test = pd.merge(test, relationship_por, on='relationship', how='left')

#### del_ocup_hpw_mean = 직업별 평균근무시간 - 해당직업에 근무하는 개인의 근무시간

In [494]:
occupation_hours_mean = train[['occupation',"hours_per_week"]].groupby('occupation',as_index=False)[['hours_per_week']].mean()

occupation_hours_mean.columns = ['occupation','ocp_hours_mean']

#hours_mean.head(5)

#ocup_hpw_mean => 직업별 평균근무시간

train= pd.merge(train,occupation_hours_mean, on='occupation', how='left')
test= pd.merge(test,occupation_hours_mean, on='occupation', how='left')

#del_ocup_hpw_mean = 직업별 평균근무시간 - 해당직업에 근무하는 개인의 근무시간
train['del_ocp_hours_mean'] = train['hours_per_week']\
                                                    - train['ocp_hours_mean']
#train.drop('ocp_hours_mean',axis =1, inplace=True)
test['del_ocp_hours_mean'] = test['hours_per_week']\
                                                    - test['ocp_hours_mean']
#test.drop('ocp_hours_mean',axis =1, inplace=True)

#### del_age_education_num_mean = 연령대별 평균 교육수준 - 해당 연령대인 개인의 교육수준

In [495]:
age_education_num_mean = train[['age','education_num']].groupby('age',as_index=False)[['education_num']].mean()

age_education_num_mean.columns = ['age', 'age_education_num_mean']
#education_num_mean => 연령대별 평균 교육수준

train= pd.merge(train,age_education_num_mean, on='age', how='left')
test= pd.merge(test,age_education_num_mean, on='age', how='left')

#del_age_education_num_mean = 연령대별 평균 교육수준 - 해당 연령대인 개인의 교육수준
train['del_age_education_num_mean'] = train['education_num']\
                                                            - train['age_education_num_mean']
#train.drop('age_education_num_mean',axis =1, inplace=True)
test['del_age_education_num_mean'] = test['education_num']\
                                                            - test['age_education_num_mean']
#test.drop('age_education_num_mean',axis =1, inplace=True)

### 데이터 쪼개기

In [61]:
train.income = train.income.apply(lambda x : 1 if x =='>50K' else 0)

In [63]:
train.drop('education_num', axis=1, inplace=True)
test.drop('education_num', axis=1, inplace=True)

In [64]:
train.sex = train.sex.apply(lambda x : 1 if x == 'Male' else 0)
test.sex = test.sex.apply(lambda x : 1 if x == 'Male' else 0)

#### train  workclass가 private / race가 white / native-country 가 unitedstates

In [65]:
train_p_w_n = train[(train.workclass == 'Private') & (train.race == 'White')&(train.native_country == 'United-States')]

In [66]:
train_p_w_n_index = train_p_w_n.index.tolist()

In [67]:
train_index_list = train.index.tolist()

In [68]:
not_train_p_w_n_index  = set(train_index_list) - set(train_p_w_n_index)

In [69]:
not_train_p_w_n = train.loc[not_train_p_w_n_index, :]

#### test  workclass가 private / race가 white / native-country 가 unitedstates

In [70]:
test_p_w_n = test[(test.workclass == 'Private') & (test.race == 'White')&(test.native_country == 'United-States')]

In [71]:
test_p_w_n_index = test_p_w_n.index.tolist()

In [72]:
test_index_list = test.index.tolist()

In [73]:
not_test_p_w_n_index  = set(test_index_list) - set(test_p_w_n_index)

In [74]:
not_test_p_w_n = test.loc[not_test_p_w_n_index, :]

### 전처리

#### workclass가 private / race가 white / native-country 가 unitedstates

In [77]:
train_p_w_n.head(1)

,age,fnlwgt,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,income,edu_portion
0,40,168538,HS-grad,Married-civ-spouse,Sales,Husband,1,0,0,60,1,0.158544


In [76]:
train_p_w_n.drop(['workclass','wor_portion', 'race', 'race_portion', 'native_country'\
                  ,'native_portion'], axis=1, inplace=True)

/Users/pym/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [79]:
test_p_w_n.head(1)

,age,fnlwgt,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,edu_portion
0,28,67661,Some-college,Never-married,Adm-clerical,Other-relative,0,0,0,40,0.192586


In [78]:
test_p_w_n.drop(['workclass', 'wor_portion', 'race', 'race_portion', 'native_country'\
                  ,'native_portion'], axis=1, inplace=True)

/Users/pym/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### workclass가 private / race가 white / native-country 가 unitedstates 아닌경우

not_train_p_w_n.drop(['education','marital_status','relationship','occupation'], axis=1, inplace=True)

not_train_p_w_n.drop(['race','workclass'], axis=1, inplace=True)

In [80]:
not_train_p_w_n.head(1)

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,edu_portion,wor_portion,race_portion,native_portion
4,24,Private,122234,Some-college,Never-married,Adm-clerical,Not-in-family,Black,0,0,0,20,?,0,0.192586,0.220328,0.121971,0.234649


not_test_p_w_n.drop(['education','marital_status','relationship','occupation'], axis=1, inplace=True)

not_test_p_w_n.drop(['race','workclass'], axis=1, inplace=True)

In [81]:
not_test_p_w_n.head(1)

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,edu_portion,wor_portion,race_portion,native_portion
1,40,Self-emp-inc,37869,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,50,United-States,0.158544,0.565564,0.257271,0.247315


### 인코딩

In [82]:
y_train_p_w_n = train_p_w_n['income']
X_train_p_w_n = train_p_w_n.drop('income', axis=1) 

In [83]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
LE_encoder = OrdinalEncoder(list(train_p_w_n.columns))
# train, test 데이터에 인코딩 적용
X_train_p_w_n = LE_encoder.fit_transform(X_train_p_w_n)

test_p_w_n = LE_encoder.fit_transform(test_p_w_n)

/Users/pym/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [84]:
y_not_train_p_w_n = not_train_p_w_n['income']
X_not_train_p_w_n = not_train_p_w_n.drop('income', axis=1)

In [85]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
n_LE_encoder = OrdinalEncoder(list(not_train_p_w_n.columns))
# train, test 데이터에 인코딩 적용
X_not_train_p_w_n = n_LE_encoder.fit_transform(X_not_train_p_w_n)

not_test_p_w_n = n_LE_encoder.fit_transform(not_test_p_w_n)

/Users/pym/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


X_not_train_p_w_n.drop('native_country', axis=1, inplace =True)
not_test_p_w_n.drop('native_country', axis=1, inplace =True)

In [86]:
X_train_p_w_n.head(1)

,age,fnlwgt,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,edu_portion
0,40,168538,1,1,1,1,1,0,0,60,0.158544


In [87]:
test_p_w_n.head(1)

,age,fnlwgt,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,edu_portion
0,28,67661,3,2,8,6,0,0,0,40,0.192586


In [88]:
X_not_train_p_w_n.head(1)

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,edu_portion,wor_portion,race_portion,native_portion
4,24,1,122234,1,1,1,1,1,0,0,0,20,1,0.192586,0.220328,0.121971,0.234649


In [89]:
not_test_p_w_n.head(1)

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,edu_portion,wor_portion,race_portion,native_portion
1,40,7,37869,2,2,11,2,2,1,0,0,50,2.0,0.158544,0.565564,0.257271,0.247315


## 모델링

### workclass 가 private / race가 white

#### lgbm

In [90]:
import gc
from lightgbm import LGBMClassifier
columns = X_train_p_w_n.columns
feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
y_preds = np.zeros(test_p_w_n.shape[0])

lgbm_clf = LGBMClassifier(objective='binary', verbose=400, random_state=91)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

fold_n = 5
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
skf.get_n_splits(X_train_p_w_n, y_train_p_w_n)
print(skf)
StratifiedKFold(n_splits=fold_n, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X_train_p_w_n, y_train_p_w_n):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_train_p_w_n.iloc[train_index], X_train_p_w_n.iloc[test_index]
    y_train, y_test = y_train_p_w_n.iloc[train_index], y_train_p_w_n.iloc[test_index]
     
    evals = [(X_train, y_train), (X_test, y_test)]
    lgbm_clf.fit(X_train, y_train, eval_metric='f1', eval_set=evals, verbose=True)
    
    feature_importances[f'skfold_{fold_n + 1}'] = lgbm_clf.feature_importances_
        
    y_preds += lgbm_clf.predict(test_p_w_n) / fold_n
    
    del X_train, X_test, y_train, y_test
    gc.collect()

StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
TRAIN: [    0     1     2 ... 14198 14200 14201] TEST: [   26    27    32 ... 14176 14183 14199]
[LightGBM] [Info] Number of positive: 3113, number of negative: 9668
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.119459
[LightGBM] [Debug] init for col-wise cost 0.000044 seconds, init for row-wise cost 0.000713 seconds
[LightGBM] [Debug] col-wise cost 0.000129 seconds, row-wise cost 0.000162 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001004 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 581
[LightGBM] [Info] Number of data points in the train set: 12781, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.243565 -> initscore=-1.133235
[LightGBM] [Info] Start training from score -1.133235
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[1]	training's binary

In [589]:
saple_sub_p_w_n = sample_submission.loc[test_p_w_n_index,:]

In [590]:
saple_sub_p_w_n['prediction'] = y_preds

for ix, row in saple_sub_p_w_n.iterrows():
    if row['prediction'] > 0.5:
        saple_sub_p_w_n.loc[ix, 'prediction'] = 1
    else:
        saple_sub_p_w_n.loc[ix, 'prediction'] = 0
        
saple_sub_p_w_n = saple_sub_p_w_n.astype({"prediction": int})

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X_train_p_w_n, y_train_p_w_n, test_size = 0.2)

print(X_train.shape, X_test.shape)

lgbm_clf.fit(X_train, y_train)
y_preds = lgbm_clf.predict(X_test)

get_clf_eval(y_test, y_preds)

(11361, 11) (2841, 11)
[LightGBM] [Info] Number of positive: 2768, number of negative: 8593
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.119655
[LightGBM] [Debug] init for col-wise cost 0.000053 seconds, init for row-wise cost 0.000840 seconds
[LightGBM] [Debug] col-wise cost 0.000153 seconds, row-wise cost 0.000132 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 575
[LightGBM] [Info] Number of data points in the train set: 11361, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.243641 -> initscore=-1.132823
[LightGBM] [Info] Start training from score -1.132823
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves

valid_idx = X_valid.index.tolist()

valid_train = train.loc[valid_idx, :]

valid_train['income'] = y_valid

valid_train['pred'] = y_preds

not_good= valid_train.loc[(valid_train.income - valid_train.pred)!= 0,:]

not_good_1 = not_good[not_good.pred == 0] #실제 수입이 높은사람을 수입이 낮은 사람이라고 잘못 예측한 경우

not_good_1.drop(['workclass','wor_portion','race_portion','native_portion','race','native_country'],axis=1, inplace=True)

In [126]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
    
    print('오차 행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, auc))

In [230]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test = train_test_split(X_train_p_w_n, y_train_p_w_n,
                                                   test_size = 0.2, random_state = 156)

In [220]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

In [227]:
#xgboost
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
#lgbm
lgbm_clf = lgb.LGBMClassifier(objective='binary', verbose=400, random_state=91)
# 결정 트리
dt_clf = DecisionTreeClassifier()
# 에이다부스트
ada_clf = AdaBoostClassifier(n_estimators = 100)
#랜덤포레스트
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)

In [224]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings    

#### lgbm 하이퍼 파라미터 튜닝

In [245]:
#lgbm
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            n_jobs = -1
        )
        
        model.fit(x_train, y_train,n_jobs=-1)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [246]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=X_train_p_w_n, y_data=y_train_p_w_n, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.9209   |  0.0708   |  0.08152  |  790.0    |  304.6    |  1.931    |  48.95    |  0.4062   |
|  2        |  0.9168   |  0.7578   |  0.009006 |  328.4    |  639.9    |  4.599    |  10.92    |  0.6635   |
|  3        |  0.9221   |  0.6787   |  0.09504  |  299.5    |  640.8    |  3.833    |  20.02    |  0.9427   |
|  4        |  0.9227   |  0.9299   |  0.09484  |  394.5    |  361.0    |  6.648    |  2.116    |  0.2322   |
|  5        |  0.9239   |  0.4301   |  0.007886 |  788.7    |  876.7    |  1.504    |  5.057    |  0.2709   |
|  6        |  0.918    |  0.859    |  0.05304  |  792.1    |  884.0    |  2.393    |  6.455    |  0.8553   |
|  7        |  0.9221   |  0.7687   |  0.05415  |  396.2    |  129.7    |  2.674    |  23.35    |  0.3523   |
|  8      

#### xgboost 하이퍼 파라미터 튜닝

In [269]:
#xgb
def xgb_cv(min_child_weight,eta,max_depth,num_leaves, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = xgb.XGBClassifier(
            booster="gbtree", #트리,회귀(gblinear) 트리가 항상 더 좋은 성능을 내기 때문에 수정할 필요없다고한다.
            silent=False, #running message출력안한다.
            
            min_child_weight=min_child_weight, #값이 높아지면 under-fitting 되는 경우가 있다. cv를 통해 튜닝되어야 한다.
            max_depth=int(max_depth), # 트리의 최대 깊이를 정의함. 루트에서 가장 긴 노드의 거리 
                        # 8이면 중요변수에서 결론까지 변수가 9개거친다.
            
            gamma =0, #노드가 split 되기 위한 loss fuction의 값이 감소하는 최소값을 정의한다.
                     #gamma값이 높아질 수록 알고리즘은 보수적으로 변하고, loss fuction의 정의에 따라
                     #적정값이 달라지기 때문에 반드시 튜닝
                        
            objective = 'binary:logistic',
            # 'reg:linear', 'binary:logistic','multi:softmax',
            # 'multi:softprob' #4가지 존재
            # 회기의 경우 'reg', binary분류의 경우 'binary',
            # 다중분류경우 'multi'- 분류된 class를 return하는 경우 'softmax'
            # 각 class에 속할 확률을 return하는 경우 'softprob'
            num_leaves = int(num_leaves), 
            learning_rate = max(eta, 0), 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), #트리를 생성할때 훈련 데이터에서 변수를 샘플링 해주는 비율, 보통 0.6~0.9
            colsample_bylevel = np.clip(colsample_bylevel, 0, 1),# 트리의 레벨별로 훈련 데이터의 변수를 샘플링 해주는 비율 보통 0.6~0.9
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            n_jobs=-1,
            scoring="roc_auc"
        )
        
        model.fit(x_train, y_train,n_jobs=-1)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [270]:
# 모델과 관련없는 변수 고정
func_fixed = partial(xgb_cv, x_data=X_train_p_w_n, y_data=y_train_p_w_n, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
xgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        "eta": (0.001, 0.4),
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'colsample_bylevel': (0, 1),      # colsample_bylevel, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
        "max_depth":(3, 10),             # max_depth,        범위(3~10)
        "min_child_weight" : (1, 10),     # min_child_weight  범위(1,10)
        "max_depth": (1, 2000)
    }, 
    random_state=4321                    # 시드 고정
)
xgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  43       |  0.923    |  0.5561   |  0.0934   |  254.6    |  304.8    |  7.74     |  4.987    |  0.1745   |
|  44       |  0.9195   |  0.4894   |  0.01589  |  338.4    |  682.5    |  3.48     |  21.75    |  0.5656   |
|  45       |  0.914    |  0.775    |  0.006819 |  179.3    |  761.0    |  3.119    |  2.508    |  0.6902   |


KeyboardInterrupt: 

#### randomforest 하이퍼 파라미터 튜닝

In [341]:
#rf
def rf_cv(max_depth,min_samples_split, n_estimators, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = RandomForestClassifier(                
            n_estimators=int(max(n_estimators,0)),
            max_depth=int(max(max_depth,1)),
            min_samples_split=int(max(min_samples_split,2)), 
            class_weight="balanced",
            n_jobs=-1,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [342]:
# 모델과 관련없는 변수 고정
func_fixed = partial(rf_cv, x_data=X_train_p_w_n, y_data=y_train_p_w_n, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
rfBO = BayesianOptimization(
    func_fixed, 
    {

        "n_estimators": (10, 1000),      # n_estimators,     범위(16~1024)
        "max_depth": (1, 150),
        "min_samples_split": (2, 10),    # max_depth,        범위(3~10)
        "max_depth": (1, 2000)
    }, 
    random_state=4321                    # 시드 고정
)
rfBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------


KeyboardInterrupt: 

#### adaboost 하이퍼 파라미터 튜닝

In [347]:
#rf
def ada_cv(eta, n_estimators, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = AdaBoostClassifier(                
            n_estimators=int(max(n_estimators,0)),
            learning_rate = max(eta, 0),
        
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [348]:
# 모델과 관련없는 변수 고정
func_fixed = partial(ada_cv, x_data=X_train_p_w_n, y_data=y_train_p_w_n, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
adaBO = BayesianOptimization(
    func_fixed, 
    {
        "eta": (0.001, 0.4),
        "n_estimators": (10, 1000),      # n_estimators,     범위(16~1024)
    }, 
    random_state=4321                    # 시드 고정
)
adaBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   |    eta    | n_esti... |
-------------------------------------------------
|  1        |  0.9125   |  0.02925  |  816.9    |
|  2        |  0.9186   |  0.3074   |  293.5    |
|  3        |  0.9178   |  0.07804  |  979.1    |
|  4        |  0.9205   |  0.1631   |  760.2    |
|  5        |  0.908    |  0.03657  |  316.8    |
|  6        |  0.9162   |  0.06869  |  761.4    |
|  7        |  0.9236   |  0.3551   |  759.1    |
|  8        |  0.866    |  0.001    |  756.0    |
|  9        |  0.9154   |  0.1602   |  290.7    |
|  10       |  0.9183   |  0.2951   |  296.3    |
|  11       |  0.924    |  0.3451   |  976.2    |
|  12       |  0.9236   |  0.2965   |  973.8    |
|  13       |  0.9065   |  0.009821 |  971.0    |
|  14       |  0.9184   |  0.2936   |  299.7    |
|  15       |  0.8863   |  0.007257 |  302.7    |
|  16       |  0.9246   |  0.3878   |  982.3    |
|  17       |  0.9227   |  0.2124   |  984.7    |
|  18       |  0.919    |  0.09563  |  987.5    |


In [213]:
xgb_wrapper.fit(X_train, y_train)
lgbm_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 2755, number of negative: 8606
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.119761
[LightGBM] [Debug] init for col-wise cost 0.000039 seconds, init for row-wise cost 0.000820 seconds
[LightGBM] [Debug] col-wise cost 0.000117 seconds, row-wise cost 0.000141 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 576
[LightGBM] [Info] Number of data points in the train set: 11361, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242496 -> initscore=-1.139042
[LightGBM] [Info] Start training from score -1.139042
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree wit

AdaBoostClassifier(n_estimators=100)

In [214]:
rf_pred = rf_clf.predict(X_test)
xgb_pred = xgb_wrapper.predict(X_test)
lgbm_pred = lgbm_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

In [215]:
get_clf_eval(y_test, rf_pred)
get_clf_eval(y_test, xgb_pred)
get_clf_eval(y_test, lgbm_pred)
get_clf_eval(y_test, dt_pred)
get_clf_eval(y_test, ada_pred)

오차 행렬
[[2039   98]
 [ 294  410]]
정확도 : 0.8620, 정밀도 : 0.8071, 재현율 : 0.5824, F1 : 0.6766, AUC : 0.7683
오차 행렬
[[2030  107]
 [ 247  457]]
정확도 : 0.8754, 정밀도 : 0.8103, 재현율 : 0.6491, F1 : 0.7208, AUC : 0.7995
오차 행렬
[[2022  115]
 [ 235  469]]
정확도 : 0.8768, 정밀도 : 0.8031, 재현율 : 0.6662, F1 : 0.7283, AUC : 0.8062
오차 행렬
[[2046   91]
 [ 542  162]]
정확도 : 0.7772, 정밀도 : 0.6403, 재현율 : 0.2301, F1 : 0.3386, AUC : 0.5938
오차 행렬
[[1855  282]
 [ 274  430]]
정확도 : 0.8043, 정밀도 : 0.6039, 재현율 : 0.6108, F1 : 0.6073, AUC : 0.7394
오차 행렬
[[2029  108]
 [ 269  435]]
정확도 : 0.8673, 정밀도 : 0.8011, 재현율 : 0.6179, F1 : 0.6977, AUC : 0.7837


In [216]:
#개별 알고리즘으로부터 예측된 예측값을 칼럼 레벨로 옆으로 붙여서 피처 값으로 만들어
# 최종 메타 모델인 로지스틱 회귀에서 학습 데이터로 다시 사용

# 반환된 예측 데이터 세트는 1차원 형태의 ndarray이므로
# 먼저 반환된 예측 결과를 행 형태로 붙인 뒤,
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred,xgb_pred,lgbm_pred])
print(pred.shape)

# 넘파이의 transpose()를 이용해 행과 열 위치를 바꾼 ndarray로 변환
# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만든다
pred = np.transpose(pred)
print(pred.shape)

(6, 2841)
(2841, 6)


In [217]:
# 최종 메타 모델인 로지스틱 회귀를 학습
lr_final = LogisticRegression(C = 10)
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

get_clf_eval(y_test, final)
# → 과적합 발생 가능

오차 행렬
[[2034  103]
 [ 253  451]]
정확도 : 0.8747, 정밀도 : 0.8141, 재현율 : 0.6406, F1 : 0.7170, AUC : 0.7962


### workclass 가 private / race가 white 아닌경우

#### lgbm

In [146]:
import gc
from lightgbm import LGBMClassifier
columns = X_not_train_p_w_n.columns
feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
not_y_preds = np.zeros(not_test_p_w_n.shape[0])

lgbm_clf = LGBMClassifier(objective='binary', verbose=400, random_state=91)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

fold_n = 5
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
skf.get_n_splits(X_not_train_p_w_n, y_not_train_p_w_n)
print(skf)
StratifiedKFold(n_splits=fold_n, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X_not_train_p_w_n, y_not_train_p_w_n):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_not_train_p_w_n.iloc[train_index], X_not_train_p_w_n.iloc[test_index]
    y_train, y_test = y_not_train_p_w_n.iloc[train_index], y_not_train_p_w_n.iloc[test_index]
     
    evals = [(X_train, y_train), (X_test, y_test)]
    lgbm_clf.fit(X_train, y_train, eval_metric='f1', eval_set=evals, verbose=True)
    
    feature_importances[f'skfold_{fold_n + 1}'] = lgbm_clf.feature_importances_
        
    not_y_preds += lgbm_clf.predict(not_test_p_w_n) / fold_n
    
    del X_train, X_test, y_train, y_test
    gc.collect()

StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
TRAIN: [    0     1     2 ... 11844 11845 11846] TEST: [   10    13    28 ... 11816 11830 11837]
[LightGBM] [Info] Number of positive: 2562, number of negative: 8100
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.855163
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.234765
[LightGBM] [Debug] init for col-wise cost 0.000860 seconds, init for row-wise cost 0.001009 seconds
[LightGBM] [Debug] col-wise cost 0.000207 seconds, row-wise cost 0.000130 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001197 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 692
[LightGBM] [Info] Number of data points in the train set: 10662, number of used features: 16
[LightGBM] [Info] [binary:BoostFromS

In [598]:
saple_sub_not_p_w_n = sample_submission.loc[not_test_p_w_n_index,:]

In [599]:
saple_sub_not_p_w_n['prediction'] = not_y_preds

for ix, row in saple_sub_not_p_w.iterrows():
    if row['prediction'] > 0.5:
        saple_sub_not_p_w_n.loc[ix, 'prediction'] = 1
    else:
        saple_sub_not_p_w_n.loc[ix, 'prediction'] = 0
        
saple_sub_not_p_w_n = saple_sub_not_p_w_n.astype({"prediction": int})

In [154]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X_not_train_p_w_n, y_not_train_p_w_n, test_size = 0.33)

print(X_train.shape, X_test.shape)

lgbm_clf.fit(X_train, y_train)
y_preds = lgbm_clf.predict(X_test)
get_clf_eval(y_test, y_preds)


(7937, 17) (3910, 17)
[LightGBM] [Info] Number of positive: 1926, number of negative: 6011
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.850857
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.220042
[LightGBM] [Debug] init for col-wise cost 0.001766 seconds, init for row-wise cost 0.001408 seconds
[LightGBM] [Debug] col-wise cost 0.000280 seconds, row-wise cost 0.000186 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 686
[LightGBM] [Info] Number of data points in the train set: 7937, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242661 -> initscore=-1.138146
[LightGBM] [Info] Start training from score -1.138146
[LightGBM] [Debug] Trained a tree wit

#### xgboost

In [161]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test = train_test_split(X_not_train_p_w_n, y_not_train_p_w_n,
                                                   test_size = 0.2, random_state = 156)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [162]:
# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)

In [163]:
get_clf_eval(y_test, w_preds)

오차 행렬
[[1689  122]
 [ 194  365]]
정확도 : 0.8667, 정밀도 : 0.7495, 재현율 : 0.6530, F1 : 0.6979, AUC : 0.7928


In [ ]:
# 예측 확률이 0.5보다 크면 1, 그렇지 않으면 0으로 예측값을 결정해
# 리스트 객체인 preds에 저장
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시 : ', preds[:10])

#### rf

In [169]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test = train_test_split(X_not_train_p_w_n, y_not_train_p_w_n,
                                                   test_size = 0.2, random_state = 156)

In [170]:
from sklearn.ensemble import RandomForestClassifier
# 랜덤 포레스트 학습 및 별도의 테스트 세트로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy))

랜덤 포레스트 정확도 : 0.8532


In [193]:

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [100],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행

# 멀티 코어 환경에서는 
# RandomForestClassifier 생성자와 GridSearchCV 생성 시 n_jobs=-1 파라미터를 추가하면
# 모든 CPU 코어를 이용해 학습할 수 있다
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터 : 
 {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도 : 0.8611


In [195]:
# n_estimators를 300으로 증가시키고, 
# 최적화 하이퍼 파라미터로 다시 RandomForestClassifier를 학습시킨 뒤에
# 별도의 테스트 데이터 세트에서 예측 성능 측정

rf_clf1 = RandomForestClassifier(n_estimators = 100,
                                max_depth = 10,
                                min_samples_leaf= 8,
                                min_samples_split = 8,
                                random_state = 0)
rf_clf1.fit(X_train, y_train)
r_pred = rf_clf1.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, r_pred)))

예측 정확도 : 0.8506


In [176]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [177]:
#xgboost
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
#lgbm
lgbm_clf = LGBMClassifier(objective='binary', verbose=400, random_state=91)
# KNN
knn_clf = KNeighborsClassifier(n_neighbors = 4)
# 결정 트리
dt_clf = DecisionTreeClassifier()
# 에이다부스트
ada_clf = AdaBoostClassifier(n_estimators = 100)

In [188]:
xgb_wrapper.fit(X_train, y_train)
lgbm_clf.fit(X_train, y_train)
knn_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 2287, number of negative: 7190
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.854595
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.221030
[LightGBM] [Debug] init for col-wise cost 0.000974 seconds, init for row-wise cost 0.001226 seconds
[LightGBM] [Debug] col-wise cost 0.000227 seconds, row-wise cost 0.000161 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 704
[LightGBM] [Info] Number of data points in the train set: 9477, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.241321 -> initscore=-1.145450
[LightGBM] [Info] Start training from score -1.145450
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_

AdaBoostClassifier(n_estimators=100)

In [189]:
xgb_pred = xgb_wrapper.predict(X_test)
lgbm_pred = lgbm_clf.predict(X_test)
knn_pred = knn_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

In [196]:
get_clf_eval(y_test, rf_pred)
get_clf_eval(y_test, xgb_pred)
get_clf_eval(y_test, lgbm_pred)
get_clf_eval(y_test, knn_pred)
get_clf_eval(y_test, dt_pred)
get_clf_eval(y_test, ada_pred)

오차 행렬
[[1697  114]
 [ 240  319]]
정확도 : 0.8506, 정밀도 : 0.7367, 재현율 : 0.5707, F1 : 0.6431, AUC : 0.7539
오차 행렬
[[1689  122]
 [ 194  365]]
정확도 : 0.8667, 정밀도 : 0.7495, 재현율 : 0.6530, F1 : 0.6979, AUC : 0.7928
오차 행렬
[[1692  119]
 [ 196  363]]
정확도 : 0.8671, 정밀도 : 0.7531, 재현율 : 0.6494, F1 : 0.6974, AUC : 0.7918
오차 행렬
[[1746   65]
 [ 420  139]]
정확도 : 0.7954, 정밀도 : 0.6814, 재현율 : 0.2487, F1 : 0.3644, AUC : 0.6064
오차 행렬
[[1551  260]
 [ 211  348]]
정확도 : 0.8013, 정밀도 : 0.5724, 재현율 : 0.6225, F1 : 0.5964, AUC : 0.7395
오차 행렬
[[1671  140]
 [ 195  364]]
정확도 : 0.8586, 정밀도 : 0.7222, 재현율 : 0.6512, F1 : 0.6849, AUC : 0.7869


In [197]:
#개별 알고리즘으로부터 예측된 예측값을 칼럼 레벨로 옆으로 붙여서 피처 값으로 만들어
# 최종 메타 모델인 로지스틱 회귀에서 학습 데이터로 다시 사용

# 반환된 예측 데이터 세트는 1차원 형태의 ndarray이므로
# 먼저 반환된 예측 결과를 행 형태로 붙인 뒤,
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred,xgb_pred,lgbm_pred])
print(pred.shape)

# 넘파이의 transpose()를 이용해 행과 열 위치를 바꾼 ndarray로 변환
# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만든다
pred = np.transpose(pred)
print(pred.shape)

(6, 2370)
(2370, 6)


In [198]:
# 최종 메타 모델인 로지스틱 회귀를 학습
lr_final = LogisticRegression(C = 10)
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

get_clf_eval(y_test, final)
# → 과적합 발생 가능

오차 행렬
[[1706  105]
 [ 214  345]]
정확도 : 0.8654, 정밀도 : 0.7667, 재현율 : 0.6172, F1 : 0.6838, AUC : 0.7796


### 결과 합치기

In [601]:
sample_submission = pd.concat([saple_sub_p_w_n, saple_sub_not_p_w_n],axis=0).sort_values(by='id')

In [603]:
sample_submission.to_csv('submission_split_p_w_junchuri_n++all_portion++++++.csv', index=False)